In [34]:
!pip -q install --upgrade beautifulsoup4
from bs4 import BeautifulSoup

In [35]:
import pandas as pd 
import numpy as np
import requests

In [36]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(wiki, 'html5lib')

postcodes_dict = {} 

table = soup.find("table", "wikitable sortable")

for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        postcode = cells[0].find(text=True)
        borough = cells[1].find(text=True)
        neighborhood = cells[2].find(text=True)
        
        #Ignore cells with a borough that is Not assigned.
        if ( borough != "Not assigned" ):
           
        #More than one neighborhood can exist in one postal code area. hese two rows will be combined into one row with the neighborhoods separated with a comma.
            try:
                neighborhood_old = postcodes_dict[postcode]['neighborhoods']
                  
            except:
                postcodes_dict[postcode] = {}
                postcodes_dict[postcode]['borough'] = borough
                postcodes_dict[postcode]['neighborhoods'] = neighborhood
                #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
                if ( neighborhood == "Not assigned\n"):
                    neighborhood = borough
                else:
                    neighborhood = neighborhood
            
            else:
                neighborhood_new = neighborhood_old + ', ' + neighborhood
                postcodes_dict[postcode]['neighborhoods'] = neighborhood_new
                
            
                                       
            
          

In [37]:
postcodes_dict

{'M1B': {'borough': 'Scarborough', 'neighborhoods': 'Rouge, Malvern'},
 'M1C': {'borough': 'Scarborough',
  'neighborhoods': 'Highland Creek, Rouge Hill, Port Union'},
 'M1E': {'borough': 'Scarborough',
  'neighborhoods': 'Guildwood\n, Morningside, West Hill'},
 'M1G': {'borough': 'Scarborough', 'neighborhoods': 'Woburn'},
 'M1H': {'borough': 'Scarborough', 'neighborhoods': 'Cedarbrae'},
 'M1J': {'borough': 'Scarborough', 'neighborhoods': 'Scarborough Village'},
 'M1K': {'borough': 'Scarborough',
  'neighborhoods': 'East Birchmount Park\n, Ionview, Kennedy Park'},
 'M1L': {'borough': 'Scarborough',
  'neighborhoods': 'Clairlea, Golden Mile, Oakridge'},
 'M1M': {'borough': 'Scarborough',
  'neighborhoods': 'Cliffcrest, Cliffside, Scarborough Village West\n'},
 'M1N': {'borough': 'Scarborough',
  'neighborhoods': 'Birch Cliff, Cliffside West\n'},
 'M1P': {'borough': 'Scarborough',
  'neighborhoods': 'Dorset Park, Scarborough Town Centre, Wexford Heights'},
 'M1R': {'borough': 'Scarboroug

In [38]:

columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto = pd.DataFrame(columns=columns)

for ind, postcode in enumerate(postcodes_dict):
    borough = postcodes_dict[postcode]['borough']
    neighborhood = postcodes_dict[postcode]['neighborhoods']
    toronto = toronto.append({"PostalCode": postcode, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

In [39]:
toronto.shape[0]

103

In [40]:
toronto

,PostalCode,Borough,Neighborhood
0,M4Y,Downtown Toronto,Church and Wellesley
1,M5G,Downtown Toronto,Central Bay Street
2,M5N,Central Toronto,Roselawn
3,M4M,East Toronto,Studio District
4,M4R,Central Toronto,North Toronto West
5,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"
6,M5E,Downtown Toronto,Berczy Park
7,M6K,West Toronto,"Brockton\n, Exhibition Place, Parkdale Village"
8,M2M,North York,"Newtonbrook, Willowdale"
9,M2P,North York,York Mills West


In [41]:
Geocoder = pd.read_csv("https://cocl.us/Geospatial_data")
Geocoder = Geocoder.groupby('Postal Code').first()
Coordinates = toronto.join(Geocoder,on='PostalCode',how="left")
Coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
2,M5N,Central Toronto,Roselawn,43.711695,-79.416936
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
5,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M6K,West Toronto,"Brockton\n, Exhibition Place, Parkdale Village",43.636847,-79.428191
8,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
9,M2P,North York,York Mills West,43.752758,-79.400049


In [42]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Coordinates['Borough'].unique()),
        Coordinates.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [43]:
import json
from pandas.io.json import json_normalize

In [44]:
!conda install -c conda-forge folium=0.5.0 --yes
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [69]:
Coordinates_Toronto = Coordinates[Coordinates['Borough'].str.contains('Toronto')]

In [70]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Coordinates_Toronto['Borough'].unique()),
        Coordinates_Toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


In [45]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

CLIENT_ID = '3U4HCOPRSFKFWXA5YKGHUB5SHMRLYDLAGH1B4A53CRWG4OFY' # your Foursquare ID
CLIENT_SECRET = 'XBKCPYV5V3LBLMM4GSVCRWCXPWX5OZ4I14Z5LG0YYNYNB5JA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Your credentails:
CLIENT_ID: 3U4HCOPRSFKFWXA5YKGHUB5SHMRLYDLAGH1B4A53CRWG4OFY
CLIENT_SECRET:XBKCPYV5V3LBLMM4GSVCRWCXPWX5OZ4I14Z5LG0YYNYNB5JA


In [71]:
toronto_venues = getNearbyVenues(names=Coordinates_Toronto['Neighborhood'],
                                   latitudes=Coordinates_Toronto['Latitude'],
                                   longitudes=Coordinates_Toronto['Longitude']
                                  )

Church and Wellesley
Central Bay Street

Roselawn

Studio District

North Toronto West

Chinatown, Grange Park, Kensington Market
Berczy Park
Brockton
, Exhibition Place, Parkdale Village
First Canadian Place, Underground city
Harbourfront East
, Toronto Islands, Union Station
Harbord
, University of Toronto
Ryerson, Garden District

Rosedale
Design Exchange, Toronto Dominion Centre
Business Reply Mail Processing Centre 969 Eastern

The Annex, North Midtown
, Yorkville
Dovercourt Village, Dufferin

St. James Town
Little Portugal, Trinity
Davisville

Cabbagetown, St. James Town
Harbourfront, Regent Park
The Danforth West
, Riverdale
Deer Park, Forest Hill SE
, Rathnelly, South Hill, Summerhill West

Christie

Commerce Court, Victoria Hotel

Stn A PO Boxes 25 The Esplanade

Runnymede, Swansea
Lawrence Park
Moore Park, Summerhill East

High Park, The Junction South

The Beaches
Forest Hill North, Forest Hill West

Parkdale, Roncesvalles
Adelaide, King, Richmond

Davisville North

The Beac

In [72]:
print(toronto_venues.shape)
toronto_venues.head()

(1692, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Church and Wellesley,43.66586,-79.38316,DanceLifeX Centre,43.666956,-79.385297,Dance Studio
1,Church and Wellesley,43.66586,-79.38316,Smith,43.666927,-79.381421,Breakfast Spot
2,Church and Wellesley,43.66586,-79.38316,The Alley,43.665922,-79.385567,Bubble Tea Shop
3,Church and Wellesley,43.66586,-79.38316,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant
4,Church and Wellesley,43.66586,-79.38316,Glad Day Bookshop,43.665271,-79.380785,Bookstore


In [73]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond\n",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton\n, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay\n, Island airport\n, Harbourfront West\n, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,77,77,77,77,77,77
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [74]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [75]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
toronto_onehot.shape

(1692, 239)

In [77]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond\n",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton\n, Exhibition Place, Parkdale Village",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 East...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay\n, Island airport\n, H...",0.000000,0.00,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.012987,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.012987,0.000000,0.000000,0.012987,0.000000,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.000000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.012195,0.00,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.012195,0.012195,0.000000,0.012195,0.012195,0.00


In [78]:
toronto_grouped.shape

(38, 239)

In [79]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond
----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4             Café  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Seafood Restaurant  0.03
4              Bakery  0.03


----Brockton
, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1            Café  0.11
2     Coffee Shop  0.11
3             Gym  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern
----
                venue  freq
0  Light Rail Station  0.11
1         Pizza Place  0.06
2          Restaurant  0.06
3       Burrito Place  0.06
4          Smoke Shop  0.06


----CN Tower, Bathurst Quay
, Island airport
, Harbourfront West
, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.15
1  Airport Terminal  0.15

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond\n",Coffee Shop,Thai Restaurant,Bar,Café,Steakhouse,Gym,Hotel,Bakery,American Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Pub,Steakhouse,Bakery,Italian Restaurant
2,"Brockton\n, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Burrito Place,Stadium,Bar,Caribbean Restaurant,Restaurant,Furniture / Home Store,Italian Restaurant
3,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
4,"CN Tower, Bathurst Quay\n, Island airport\n, H...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate


In [82]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [83]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Coordinates_Toronto

toronto_merged.columns = ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Dance Studio,Bubble Tea Shop,Men's Store,Café,Burger Joint,Mediterranean Restaurant
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Thai Restaurant,Sandwich Place,Spa,Ice Cream Shop
2,M5N,Central Toronto,Roselawn,43.711695,-79.416936,4,Ice Cream Shop,Garden,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Gastropub,Bookstore,Latin American Restaurant,Fish Market,Brewery
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Sporting Goods Shop,Clothing Store,Coffee Shop,Yoga Studio,Shoe Store,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Pet Store


In [84]:
import matplotlib.cm as cm
import matplotlib.colors as colors

address = 'Toronto, CA'
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Downtown Toronto,0,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
80,Central Toronto,0,Playground,Gym,Tennis Court,Park,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Electronics Store


In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East Toronto,1,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
77,Central Toronto,1,Gym / Fitness Center,Park,Swim School,Bus Line,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
95,Central Toronto,1,Sandwich Place,Burger Joint,Gym,Breakfast Spot,Park,Hotel,Clothing Store,Food & Drink Shop,Dog Run,Doner Restaurant
96,East Toronto,1,Pizza Place,Park,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Pet Store,Movie Theater,Sushi Restaurant


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Dance Studio,Bubble Tea Shop,Men's Store,Café,Burger Joint,Mediterranean Restaurant
1,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Thai Restaurant,Sandwich Place,Spa,Ice Cream Shop
3,East Toronto,2,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Gastropub,Bookstore,Latin American Restaurant,Fish Market,Brewery
4,Central Toronto,2,Sporting Goods Shop,Clothing Store,Coffee Shop,Yoga Studio,Shoe Store,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Pet Store
5,Downtown Toronto,2,Café,Vegetarian / Vegan Restaurant,Bar,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Bakery,Mexican Restaurant,Cocktail Bar,Farmers Market
6,Downtown Toronto,2,Coffee Shop,Restaurant,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Pub,Steakhouse,Bakery,Italian Restaurant
7,West Toronto,2,Breakfast Spot,Coffee Shop,Café,Burrito Place,Stadium,Bar,Caribbean Restaurant,Restaurant,Furniture / Home Store,Italian Restaurant
12,Downtown Toronto,2,Coffee Shop,Café,Hotel,Steakhouse,Deli / Bodega,Restaurant,Bakery,Seafood Restaurant,Asian Restaurant,American Restaurant
24,Downtown Toronto,2,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Fried Chicken Joint,Brewery,Restaurant,Scenic Lookout,Pizza Place
28,Downtown Toronto,2,Café,Bakery,Bar,Japanese Restaurant,Bookstore,Restaurant,Poutine Place,Beer Bar,Beer Store,Sandwich Place


In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Central Toronto,3,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,4,Ice Cream Shop,Garden,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
